In [916]:
# importação de biblitoeca
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import pandas as pd
import statistics as sts
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score
from yellowbrick.classifier import ConfusionMatrix
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
import graphviz
from sklearn.tree import export_graphviz
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz


In [917]:
# importando dataframe 
df = pd.read_csv("BankChurners.csv")
#excluindo as ultimas duas colunas
List_exclu_coluna = ["Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1", "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2",]
df = df.drop(List_exclu_coluna, axis=1)
df.shape


(10127, 21)

In [918]:
# Visualizando colunas
df.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000


In [919]:
#Traduzindo as colunas
lista_colunas = [
    'ID', 'Status_do_Cliente', 'Idade_do_Cliente', 'Gênero', 'Numero_de_Dependentes',
    'Nível_de_Educação', 'Estado_Civil', 'Renda_anual', 'Tipo_de_cartão',
    'Período_de_relacionamento_com_o_banco', 'Nº_total_de_produtos', 'Meses_Inativo',
    'Contatos_12_meses', 'Limite_de_Crédito', 'Saldo_Total_Rotativo',
    'Limite_Disponível', 'Alteração_no_valor_da_transação', 'Valor_Total_Transação',
    'Contagem_Total_de_Transações', 'Variação_Ct_Total_Q4_Q1', 'Taxa_média_de_utilização_do_cartão']


df = df.rename(columns=dict(zip(df.columns, lista_colunas)))

In [920]:
# mostrando os dados unicos de cada colunas
# Filtrar colunas do tipo string
colunas_string = df.select_dtypes(include=['object']).columns

# Mostrar dados únicos de cada coluna string
for coluna in colunas_string:
    dados_unicos = df[coluna].unique()
    print(f"Dados únicos da coluna {coluna}:")
    print(dados_unicos)
    print()

Dados únicos da coluna Status_do_Cliente:
['Existing Customer' 'Attrited Customer']

Dados únicos da coluna Gênero:
['M' 'F']

Dados únicos da coluna Nível_de_Educação:
['High School' 'Graduate' 'Uneducated' 'Unknown' 'College' 'Post-Graduate'
 'Doctorate']

Dados únicos da coluna Estado_Civil:
['Married' 'Single' 'Unknown' 'Divorced']

Dados únicos da coluna Renda_anual:
['$60K - $80K' 'Less than $40K' '$80K - $120K' '$40K - $60K' '$120K +'
 'Unknown']

Dados únicos da coluna Tipo_de_cartão:
['Blue' 'Gold' 'Silver' 'Platinum']



In [921]:
#Traduzindo dados da colunas
df['Status_do_Cliente'].replace(['Existing Customer', 'Attrited Customer'], ['Cliente Ativo', 'Cliente Inativo'], inplace=True)
df['Nível_de_Educação'].replace(['High School', 'Graduate', 'Uneducated', 'Unknown', 'College', 'Post-Graduate', 'Doctorate'], ['Ensino médio', 'Graduação', 'Sem escolaridade', 'Desconhecido', 'Faculdade', 'Pós-Graduação', 'Doutorado'], inplace=True)
df['Renda_anual'].replace(['$60K - $80K','Less than $40K','$80K - $120K','$40K - $60K','$120K +','Unknown'] , ['US$ 60 mil - US$ 80 mil','Menos de US$ 40 mil','US$ 80 mil - US$ 120 mil','US$ 40 mil - US$ 60 mil','US$ 120 mil +','Desconhecido'] , inplace=True)
df['Estado_Civil'].replace(['Married', 'Single', 'Unknown', 'Divorced'], ['Casado', 'Solteiro', 'Desconhecido', 'Divorciado'], inplace=True)



In [922]:
#Visualizando dados corrigidos
df.head()

,ID,Status_do_Cliente,Idade_do_Cliente,Gênero,Numero_de_Dependentes,Nível_de_Educação,Estado_Civil,Renda_anual,Tipo_de_cartão,Período_de_relacionamento_com_o_banco,...,Meses_Inativo,Contatos_12_meses,Limite_de_Crédito,Saldo_Total_Rotativo,Limite_Disponível,Alteração_no_valor_da_transação,Valor_Total_Transação,Contagem_Total_de_Transações,Variação_Ct_Total_Q4_Q1,Taxa_média_de_utilização_do_cartão
0,768805383,Cliente Ativo,45,M,3,Ensino médio,Casado,US$ 60 mil - US$ 80 mil,Blue,39,...,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061
1,818770008,Cliente Ativo,49,F,5,Graduação,Solteiro,Menos de US$ 40 mil,Blue,44,...,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105
2,713982108,Cliente Ativo,51,M,3,Graduação,Casado,US$ 80 mil - US$ 120 mil,Blue,36,...,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000
3,769911858,Cliente Ativo,40,F,4,Ensino médio,Desconhecido,Menos de US$ 40 mil,Blue,34,...,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760
4,709106358,Cliente Ativo,40,M,3,Sem escolaridade,Casado,US$ 60 mil - US$ 80 mil,Blue,21,...,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000


In [923]:
#Verificação dos dados nuloes
df.isnull().sum()

ID                                       0
Status_do_Cliente                        0
Idade_do_Cliente                         0
Gênero                                   0
Numero_de_Dependentes                    0
Nível_de_Educação                        0
Estado_Civil                             0
Renda_anual                              0
Tipo_de_cartão                           0
Período_de_relacionamento_com_o_banco    0
Nº_total_de_produtos                     0
Meses_Inativo                            0
Contatos_12_meses                        0
Limite_de_Crédito                        0
Saldo_Total_Rotativo                     0
Limite_Disponível                        0
Alteração_no_valor_da_transação          0
Valor_Total_Transação                    0
Contagem_Total_de_Transações             0
Variação_Ct_Total_Q4_Q1                  0
Taxa_média_de_utilização_do_cartão       0
dtype: int64

In [924]:
#verificando se há dados duplicados
df.loc[df.duplicated("ID", keep=False)]


,ID,Status_do_Cliente,Idade_do_Cliente,Gênero,Numero_de_Dependentes,Nível_de_Educação,Estado_Civil,Renda_anual,Tipo_de_cartão,Período_de_relacionamento_com_o_banco,...,Meses_Inativo,Contatos_12_meses,Limite_de_Crédito,Saldo_Total_Rotativo,Limite_Disponível,Alteração_no_valor_da_transação,Valor_Total_Transação,Contagem_Total_de_Transações,Variação_Ct_Total_Q4_Q1,Taxa_média_de_utilização_do_cartão


In [925]:
df.describe()

,ID,Idade_do_Cliente,Numero_de_Dependentes,Período_de_relacionamento_com_o_banco,Nº_total_de_produtos,Meses_Inativo,Contatos_12_meses,Limite_de_Crédito,Saldo_Total_Rotativo,Limite_Disponível,Alteração_no_valor_da_transação,Valor_Total_Transação,Contagem_Total_de_Transações,Variação_Ct_Total_Q4_Q1,Taxa_média_de_utilização_do_cartão
count,1.012700e+04,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000
mean,7.391776e+08,46.325960,2.346203,35.928409,3.812580,2.341167,2.455317,8631.953698,1162.814061,7469.139637,0.759941,4404.086304,64.858695,0.712222,0.274894
std,3.690378e+07,8.016814,1.298908,7.986416,1.554408,1.010622,1.106225,9088.776650,814.987335,9090.685324,0.219207,3397.129254,23.472570,0.238086,0.275691
min,7.080821e+08,26.000000,0.000000,13.000000,1.000000,0.000000,0.000000,1438.300000,0.000000,3.000000,0.000000,510.000000,10.000000,0.000000,0.000000
25%,7.130368e+08,41.000000,1.000000,31.000000,3.000000,2.000000,2.000000,2555.000000,359.000000,1324.500000,0.631000,2155.500000,45.000000,0.582000,0.023000
50%,7.179264e+08,46.000000,2.000000,36.000000,4.000000,2.000000,2.000000,4549.000000,1276.000000,3474.000000,0.736000,3899.000000,67.000000,0.702000,0.176000
75%,7.731435e+08,52.000000,3.000000,40.000000,5.000000,3.000000,3.000000,11067.500000,1784.000000,9859.000000,0.859000,4741.000000,81.000000,0.818000,0.503000
max,8.283431e+08,73.000000,5.000000,56.000000,6.000000,6.000000,6.000000,34516.000000,2517.000000,34516.000000,3.397000,18484.000000,139.000000,3.714000,0.999000


In [926]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 21 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   ID                                     10127 non-null  int64  
 1   Status_do_Cliente                      10127 non-null  object 
 2   Idade_do_Cliente                       10127 non-null  int64  
 3   Gênero                                 10127 non-null  object 
 4   Numero_de_Dependentes                  10127 non-null  int64  
 5   Nível_de_Educação                      10127 non-null  object 
 6   Estado_Civil                           10127 non-null  object 
 7   Renda_anual                            10127 non-null  object 
 8   Tipo_de_cartão                         10127 non-null  object 
 9   Período_de_relacionamento_com_o_banco  10127 non-null  int64  
 10  Nº_total_de_produtos                   10127 non-null  int64  
 11  Me

In [927]:
#identificando as variaveis categoricas
df.select_dtypes(include=object).describe()

,Status_do_Cliente,Gênero,Nível_de_Educação,Estado_Civil,Renda_anual,Tipo_de_cartão
count,10127,10127,10127,10127,10127,10127
unique,2,2,7,4,6,4
top,Cliente Ativo,F,Graduação,Casado,Menos de US$ 40 mil,Blue
freq,8500,5358,3128,4687,3561,9436


In [928]:
df.groupby(['Status_do_Cliente']).size()

Status_do_Cliente
Cliente Ativo      8500
Cliente Inativo    1627
dtype: int64

In [929]:
#Realizando a transformação de variaveis categoricas em numericas
df['Gênero'] = df['Gênero'].map({'M': 0, 'F': 1})
df['Status_do_Cliente'] = df['Status_do_Cliente'].map({'Cliente Inativo': 0, 'Cliente Ativo': 1})

# Utilizando o meto enconder
label_encoder = preprocessing.LabelEncoder()

df["Nível_de_Educação"] = label_encoder.fit_transform(df["Nível_de_Educação"])
df['Estado_Civil'] = label_encoder.fit_transform(df['Estado_Civil'])
df["Renda_anual"] = label_encoder.fit_transform(df['Renda_anual'])
df['Tipo_de_cartão'] = label_encoder.fit_transform(df['Tipo_de_cartão'])



In [930]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 21 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   ID                                     10127 non-null  int64  
 1   Status_do_Cliente                      10127 non-null  int64  
 2   Idade_do_Cliente                       10127 non-null  int64  
 3   Gênero                                 10127 non-null  int64  
 4   Numero_de_Dependentes                  10127 non-null  int64  
 5   Nível_de_Educação                      10127 non-null  int32  
 6   Estado_Civil                           10127 non-null  int32  
 7   Renda_anual                            10127 non-null  int32  
 8   Tipo_de_cartão                         10127 non-null  int32  
 9   Período_de_relacionamento_com_o_banco  10127 non-null  int64  
 10  Nº_total_de_produtos                   10127 non-null  int64  
 11  Me

In [931]:
# Removendo coluna ID
df = df.drop(['ID'] , axis=1)

In [932]:
# Separar os dados em Dependente e Independente
X = df.drop(['Status_do_Cliente'], axis=1)
Y = df['Status_do_Cliente']

In [933]:
# Dividir os dados em treinamento e teste (Utilizando 30% para testar e 70% para treinar)
X_treinamento, X_teste, Y_treinamento, Y_teste = train_test_split(X, Y, test_size=0.3, random_state=0)


In [934]:
# Criar uma instância do modelo de Florestas Aleatórias
modelo = RandomForestClassifier()

In [935]:
# Treinar o modelo
modelo.fit(X_treinamento, Y_treinamento)

RandomForestClassifier()

In [936]:
# Exportar a árvore de decisão
export_graphviz(modelo.estimators_[0], out_file="tree.dot", feature_names=X.columns)

In [937]:
# Fazer previsões com os dados de teste
previsoes = modelo.predict(X_teste)
previsoes



array([1, 0, 1, ..., 1, 1, 1], dtype=int64)

In [938]:
confusao = confusion_matrix(Y_teste, previsoes)
confusao

array([[ 364,   92],
       [  39, 2544]], dtype=int64)

In [939]:
# taxa de acerto
taxa_de_acerto = accuracy_score(Y_teste, previsoes)
taxa_de_acerto

0.9568937150378414